In [3]:
import pandas as pd
from collections import defaultdict
import networkx as nx
import linkpred
from sklearn import neighbors, datasets, preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.svm import SVC


df = pd.read_csv('imdb.csv')

In [4]:
# samples: 2006-2015, 2016; connected by movies
data_sample1 = defaultdict(list)
data_sample2 = defaultdict(list)  # data sample for 2016 movies 

for index, row in df.iterrows():
    actors = row['Actors'].split(',')
    year = row['Year']
    for actor in actors:
        if year < 2016:
            data_sample1[actor].append(row['Title'])
        else:
            data_sample2[actor].append(row['Title'])
            
            
G_coeffs = nx.Graph(name="Graph for coefficients")
G_y = nx.Graph(name="Graph for Y")

# generating coefficients graph
for actor in data_sample1:
    movies = data_sample1[actor]
    actor = actor.strip()
    G_coeffs.add_node(actor, actor_attributes=movies)

    for node, attrs in G_coeffs.nodes(data=True):
        if node != actor:
            for a in attrs['actor_attributes']:
                if a in movies and not G_coeffs.has_edge(actor, node):
                    G_coeffs.add_edge(actor, node, film=a)
                
# generate Y graph
for actor in data_sample2:
    movies = data_sample2[actor]
    actor = actor.strip()
    G_y.add_node(actor, actor_attributes=movies)

    for node, attrs in G_y.nodes(data=True):
        if node != actor:
            for a in attrs['actor_attributes']:
                if a in movies and not G_y.has_edge(actor, node):
                    G_y.add_edge(actor, node, film=a)
                    
indexes = []
for u, v, p in nx.jaccard_coefficient(G_coeffs):
    indexes.append(tuple((u, v)))
                    
edge_df = pd.DataFrame(index=indexes)
edge_df['Jaccard'] = [i[2] for i in nx.jaccard_coefficient(G_coeffs, edge_df.index)]
edge_df['Adamic-Adar'] = [i[2] for i in nx.adamic_adar_index(G_coeffs, edge_df.index)]
edge_df['Pref-Attach'] = [i[2] for i in nx.preferential_attachment(G_coeffs, edge_df.index)]
edge_df['Res-Alloc'] = [i[2] for i in nx.resource_allocation_index(G_coeffs, edge_df.index)]
edge_df['Y'] = [1 if G_y.has_edge(u, v) else 0 for u, v in indexes]

In [5]:
edge_df.describe()

,Jaccard,Adamic-Adar,Pref-Attach,Res-Alloc,Y
count,1.015367e+06,1.015367e+06,1.015367e+06,1.015367e+06,1.015367e+06
mean,1.431494e-03,7.189461e-03,2.400484e+01,1.489223e-03,5.229636e-04
std,1.309157e-02,5.700029e-02,3.113075e+01,1.285396e-02,2.286243e-02
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000e+00,0.000000e+00,9.000000e+00,0.000000e+00,0.000000e+00
50%,0.000000e+00,0.000000e+00,1.500000e+01,0.000000e+00,0.000000e+00
75%,0.000000e+00,0.000000e+00,2.700000e+01,0.000000e+00,0.000000e+00
max,6.666667e-01,2.164043e+00,8.700000e+02,7.500000e-01,1.000000e+00


In [14]:
edge_df.shape

(1015367, 5)

In [4]:
train, test = train_test_split(edge_df, test_size=0.3)
train_0 = train[train['Y'] == 0]
train_1 = train[train['Y'] == 1]
test_0 = test[test['Y'] == 0]
test_1 = test[test['Y'] == 1]

In [5]:
X_train = train[['Jaccard', 'Adamic-Adar', 'Pref-Attach', 'Res-Alloc']]
y_train = train[['Y']]
X_test = test[['Jaccard', 'Adamic-Adar', 'Pref-Attach', 'Res-Alloc']]
y_test = test[['Y']]

In [6]:
scaler = preprocessing.StandardScaler().fit(X_train)  # standartize data
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
y_train = y_train.values.ravel() # transform to numpy array
y_test = y_test.values.ravel()

In [7]:
knn = neighbors.KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
accuracy_knn = accuracy_score(y_test, y_pred)
recall_knn = recall_score(y_test, y_pred)
prec_knn = precision_score(y_test, y_pred)
recall_knn

/Users/irinanazarchuk/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


0.0

In [8]:
svc = SVC(kernel='linear')
svc.fit(X_train, y_train)
y_pred = svc.predict(X_test)
accuracy_svm = accuracy_score(y_test, y_pred)
recall_svm = recall_score(y_test, y_pred)
recall_svm

0.0

In [ ]:
# train on subsample
data_class_0 = train[train['Y'] == 0][:1000]
data_class_1 = train[train['Y'] == 1]
merged_data = data_class_0.append(data_class_1)
X_train = merged_data[['Jaccard', 'Adamic-Adar', 'Pref-Attach', 'Res-Alloc']]
y_train = merged_data[['Y']]
X_test = test[['Jaccard', 'Adamic-Adar', 'Pref-Attach', 'Res-Alloc']]
y_test = test[['Y']]

scaler = preprocessing.StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
y_train = y_train.values.ravel()
y_test = y_test.values.ravel()

svc = SVC(kernel='linear')
svc.fit(X_train, y_train)
y_pred = svc.predict(X_test)
recall_svm = recall_score(y_test, y_pred)

In [9]:
# with added centralities features
indexes = []
degree_src = []  # degree centrality
degree_dst = []
clos_src = []  # closenness centrality
clos_dst = []
betw_src = []  # betweenness centrality
betw_dst = []

d = nx.betweenness_centrality(G_coeffs, normalized=True)

for u, v, p in nx.jaccard_coefficient(G_coeffs):
    indexes.append(tuple((u, v)))
    degree_src.append(nx.degree(G_coeffs, u))
    degree_dst.append(nx.degree(G_coeffs, v))
    clos_src.append(nx.closeness_centrality(G_coeffs, u))
    clos_dst.append(nx.closeness_centrality(G_coeffs, v))
    betw_src.append(d[u])
    betw_dst.append(d[v])
                    
edge_df2 = pd.DataFrame(index=indexes)
edge_df2['Jaccard'] = [i[2] for i in nx.jaccard_coefficient(G_coeffs, edge_df2.index)]
edge_df2['Adamic-Adar'] = [i[2] for i in nx.adamic_adar_index(G_coeffs, edge_df2.index)]
edge_df2['Pref-Attach'] = [i[2] for i in nx.preferential_attachment(G_coeffs, edge_df2.index)]
edge_df2['Res-Alloc'] = [i[2] for i in nx.resource_allocation_index(G_coeffs, edge_df2.index)]
edge_df2['Degree_centrality'] = [min(i, j) for i, j in zip(degree_src, degree_dst)]
edge_df2['Closeness_centrality'] = [min(i, j) for i, j in zip(clos_src, clos_dst)]
edge_df2['Betweenness_ centrality'] = [min(i, j) for i, j in zip(betw_src, betw_dst)]
edge_df2['Y'] = [1 if G_y.has_edge(u, v) else 0 for u, v in indexes]

train, test = train_test_split(edge_df2, test_size=0.3)
X_train = train[['Jaccard', 'Adamic-Adar', 'Pref-Attach', 'Res-Alloc', 'Degree_centrality', 'Closeness_centrality', 'Betweenness_ centrality']]
y_train = train[['Y']]
X_test = test[['Jaccard', 'Adamic-Adar', 'Pref-Attach', 'Res-Alloc', 'Degree_centrality', 'Closeness_centrality', 'Betweenness_ centrality']]
y_test = test[['Y']]

scaler = preprocessing.StandardScaler().fit(X_train) 
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
y_train = y_train.values.ravel() 
y_test = y_test.values.ravel()

knn = neighbors.KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
accuracy_knn = accuracy_score(y_test, y_pred)
recall_knn = recall_score(y_test, y_pred)
prec_knn = precision_score(y_test, y_pred)

KeyboardInterrupt: 

In [112]:
edge_df.describe()

,Jaccard,Adamic-Adar,Pref-Attach,Res-Alloc,Deg_centrality,Y
count,1.015367e+06,1.015367e+06,1.015367e+06,1.015367e+06,1.015367e+06,1.015367e+06
mean,1.431494e-03,7.189461e-03,2.400484e+01,1.489223e-03,3.147642e+00,5.229636e-04
std,1.309157e-02,5.700029e-02,3.113075e+01,1.285396e-02,1.603312e+00,2.286243e-02
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000e+00,0.000000e+00,9.000000e+00,0.000000e+00,2.000000e+00,0.000000e+00
50%,0.000000e+00,0.000000e+00,1.500000e+01,0.000000e+00,3.000000e+00,0.000000e+00
75%,0.000000e+00,0.000000e+00,2.700000e+01,0.000000e+00,3.000000e+00,0.000000e+00
max,6.666667e-01,2.164043e+00,8.700000e+02,7.500000e-01,2.900000e+01,1.000000e+00


In [13]:
from sklearn.metrics import confusion_matrix, classification_report

In [11]:
c = confusion_matrix(y_test, y_pred)

In [12]:
c

array([[304453,      0],
       [   158,      0]])

In [17]:
target_names = ['class 0', 'class 1']
print(classification_report(y_test, y_pred, target_names=target_names))

             precision    recall  f1-score   support

    class 0       1.00      1.00      1.00    304453
    class 1       0.00      0.00      0.00       158

avg / total       1.00      1.00      1.00    304611



/Users/irinanazarchuk/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(X_train, y_train)
# clf.feature_importances_
y_pred = clf.predict(X_test)
# y_pred_log = clf.predict_log_proba(X_test)
y_pred_proba = clf.predict_proba(X_test)

# Convert predictions to a DataFrame
hof_predictions = pd.DataFrame(y_pred_proba[:,1])
hof_predictions = hof_predictions.sort_values(0, ascending=False)
hof_predictions['Probability'] = hof_predictions[0]

In [ ]:
from sklearn.linear_model import LogisticRegression

logisticRegr = LogisticRegression()
logisticRegr.fit(X_train, y_train)
y_pred = logisticRegr.predict(X_test)
score = logisticRegr.score(X_test, y_test)
cm = metrics.confusion_matrix(y_test, y_pred)

In [ ]:
from sklearn.tree import DecisionTreeClassifier

classifier = DecisionTreeClassifier()  
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)  

In [ ]:
from sklearn.decomposition import PCA

# pca = PCA(.95)  # keeping variance
pca = PCA(n_components=2)
pca.fit(X_train)
X_train = pca.transform(X_train)
X_test = pca.transform(X_test)
# pca.explained_variance_ratio_

In [ ]:
from sklearn.metrics import roc_auc_score

auc = roc_auc_score(y_test, y_pred)